In [3]:
import pandas as pd

In [4]:
# Load collected data
df = pd.read_csv("financial_news_cleaned.csv")
df

,headline,summary,created_at,ticker,headline_summary,tokens,filtered_tokens,lemmatized,impact_score,sentiment_label,sentiment_score
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again,2024-01-26,IBM,How We Missed A Winning Trade On IBM--And How ...,"['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...",0.0,Negative,0.967072
1,Investigating IBM's Standing In IT Services In...,NaN,2024-01-26,IBM,Investigating IBM's Standing In IT Services In...,"['Investigating', 'IBM', ""'s"", 'Standing', 'In...","['Investigating', 'IBM', ""'s"", 'Standing', 'In...","['Investigating', 'IBM', ""'s"", 'Standing', 'In...",0.0,Neutral,0.998521
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26,IBM,90's Tech Titans Are Outshining 'Magnificent S...,"['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...","['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...","['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...",1.2,Positive,1.000000
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25,IBM,Why Humana Shares Are Trading Lower By Over 11...,"['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...",1.9,Negative,0.999982
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...",2.0,Positive,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
6798,"Earnings Scheduled For July 30, 2020",Companies Reporting Before The Bell • Masterca...,2020-07-30,TOT,"Earnings Scheduled For July 30, 2020 Companies...","['Earnings', 'Scheduled', 'For', 'July', '30',...","['Earnings', 'Scheduled', 'For', 'July', '30',...","['Earnings', 'Scheduled', 'For', 'July', '30',...",1.8,Neutral,0.999995
6799,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"Gainers • Laredo Petroleum, Inc. (NYSE:LPI) st...",2020-06-02,TOT,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...","['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...","['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...",0.0,Positive,0.999999
6800,"Earnings Scheduled For May 5, 2020",Companies Reporting Before The Bell L3Harris T...,2020-05-05,TOT,"Earnings Scheduled For May 5, 2020 Companies R...","['Earnings', 'Scheduled', 'For', 'May', '5', '...","['Earnings', 'Scheduled', 'For', 'May', '5', '...","['Earnings', 'Scheduled', 'For', 'May', '5', '...",1.8,Neutral,0.999999
6801,"Earnings Scheduled For April 30, 2020",Companies Reporting Before The Bell McDonald&#...,2020-04-30,TOT,"Earnings Scheduled For April 30, 2020 Companie...","['Earnings', 'Scheduled', 'For', 'April', '30'...","['Earnings', 'Scheduled', 'For', 'April', '30'...","['Earnings', 'Scheduled', 'For', 'April', '30'...",1.8,Neutral,0.999999


Need features for evaluating how valuable (perhaps the word relevant is more accurate for what we're doing here) a financial news post may be, for some investor looking for stocks that will grow (this is narrowing down our criteria)

- Direct Impact on Stocks: News directly related to specific companies (e.g., earnings reports, mergers and acquisitions, major contracts, leadership changes).
- Sentiment Analysis: Gauge the overall sentiment (positive, negative, neutral) of the news and its potential influence on investor behavior.
- Recent and Immediate News: Information that is current and has immediate implications for the market.
- Forward-Looking Statements: News containing projections, forecasts, or future plans that could impact long-term stock performance.

Impact analysis

In [9]:
# Notes: The weights are subjective and could be refined based on specific 
# industry knowledge or empirical analysis of past market reactions to similar news.
impact_keywords = {
    "earnings": 1.0, "revenue": 0.8, "profit": 0.9, "loss": 1.0, "EPS": 0.7,
    "guidance": 0.9, "forecast": 0.8, "dividend": 0.6,
    "merger": 1.2, "acquisition": 1.2, "takeover": 1.1,
    "bankruptcy": 1.5, "restructuring": 1.3, "layoff": 1.1,
    "lawsuit": 1.4, "patent": 0.7, "approval": 0.8,
    "IPO": 1.0, "stock split": 0.9, "market share": 0.8,
    "CEO": 1.0, "CFO": 0.9, "resignation": 1.1,
    "expansion": 0.8, "contract": 0.7, "partnership": 0.7,
    "regulation": 1.1, "compliance": 0.8, "trade war": 1.3,
    "interest rate": 1.2, "inflation": 1.1, "economic growth": 0.9,
    "funding": 0.7, "investment": 0.8, "venture capital": 0.7,
    "debt": 1.1, "credit": 0.8, "default": 1.4,
    "product launch": 0.9, "innovation": 0.8, "R&D": 0.7,
    "shareholder": 0.7, "stakeholder": 0.6, "buyout": 1.2,
    "sustainability": 0.7, "ESG": 0.7, "environmental": 0.7,
    "litigation": 1.3, "fraud": 1.5, "scandal": 1.4,
    "data breach": 1.4, "cybersecurity": 0.9, "privacy": 0.8,
    "regulatory approval": 1.0, "antitrust": 1.2, "monopoly": 1.1
}

forecast_keywords = {
    "forecast": 1.0,
    "projection": 0.9,
    "outlook": 0.8,
    "guidance": 0.9,
    "expansion plans": 1.0,
    "growth strategy": 1.0,
    "investment plan": 0.8,
    "R&D pipeline": 0.9,
    "capital expenditure": 0.7,
    "market penetration strategy": 0.8,
    "innovation roadmap": 0.9,
    "strategic initiatives": 0.8,
    "long-term goals": 0.6,
    "product roadmap": 0.9,
    "market expansion": 0.8,
    "diversification strategy": 0.7,
    "sustainability initiatives": 0.6,
    "merger & acquisition plans": 1.1,
    "supply chain optimization": 0.7,
    "technological advancements": 0.9
}

def calculate_score(text, keywords):
    score = 0.0
    lower_text = text.lower()  # Convert text to lowercase for case-insensitive matching
    for keyword, weight in keywords.items():
        if keyword in lower_text:
            score += weight
    return score

df['impact_score'] = df['headline_summary'].apply(lambda x: ca
                                                  lculate_score(x, impact_keywords))
df['forecast_score'] = df['headline_summary'].apply(lambda x: calculate_score(x, forecast_keywords))
df

,headline,summary,created_at,ticker,headline_summary,tokens,filtered_tokens,lemmatized,impact_score,sentiment_label,sentiment_score,forecast_score
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again,2024-01-26,IBM,How We Missed A Winning Trade On IBM--And How ...,"['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...",0.0,Negative,0.967072,0.0
1,Investigating IBM's Standing In IT Services In...,NaN,2024-01-26,IBM,Investigating IBM's Standing In IT Services In...,"['Investigating', 'IBM', ""'s"", 'Standing', 'In...","['Investigating', 'IBM', ""'s"", 'Standing', 'In...","['Investigating', 'IBM', ""'s"", 'Standing', 'In...",0.0,Neutral,0.998521,0.0
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26,IBM,90's Tech Titans Are Outshining 'Magnificent S...,"['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...","['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...","['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...",1.2,Positive,1.000000,0.0
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25,IBM,Why Humana Shares Are Trading Lower By Over 11...,"['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...",1.9,Negative,0.999982,0.9
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...",2.0,Positive,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6798,"Earnings Scheduled For July 30, 2020",Companies Reporting Before The Bell • Masterca...,2020-07-30,TOT,"Earnings Scheduled For July 30, 2020 Companies...","['Earnings', 'Scheduled', 'For', 'July', '30',...","['Earnings', 'Scheduled', 'For', 'July', '30',...","['Earnings', 'Scheduled', 'For', 'July', '30',...",1.8,Neutral,0.999995,0.0
6799,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"Gainers • Laredo Petroleum, Inc. (NYSE:LPI) st...",2020-06-02,TOT,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...","['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...","['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...",0.0,Positive,0.999999,0.0
6800,"Earnings Scheduled For May 5, 2020",Companies Reporting Before The Bell L3Harris T...,2020-05-05,TOT,"Earnings Scheduled For May 5, 2020 Companies R...","['Earnings', 'Scheduled', 'For', 'May', '5', '...","['Earnings', 'Scheduled', 'For', 'May', '5', '...","['Earnings', 'Scheduled', 'For', 'May', '5', '...",1.8,Neutral,0.999999,0.0
6801,"Earnings Scheduled For April 30, 2020",Companies Reporting Before The Bell McDonald&#...,2020-04-30,TOT,"Earnings Scheduled For April 30, 2020 Companie...","['Earnings', 'Scheduled', 'For', 'April', '30'...","['Earnings', 'Scheduled', 'For', 'April', '30'...","['Earnings', 'Scheduled', 'For', 'April', '30'...",1.8,Neutral,0.999999,0.0


Sentiment analysis

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import numpy as np

# Load pre-trained model and tokenizer
model_name = "yiyanghkust/finbert-tone"  # Example FinBERT model
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Create a sentiment analysis pipeline
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Function to apply FinBERT to a batch of texts
def finbert_helper(texts):
    results = finbert(texts)
    labels = [result['label'] for result in results]
    scores = [result['score'] for result in results]
    return labels, scores

# Batch processing function
def apply_finbert_to_batch(dataframe, column_name, batch_size=20):
    num_batches = int(np.ceil(len(dataframe) / batch_size))
    labels, scores = [], []

    for i in range(num_batches):
        batch_texts = dataframe[column_name].iloc[i * batch_size:(i + 1) * batch_size].tolist()
        batch_labels, batch_scores = finbert_helper(batch_texts)
        labels.extend(batch_labels)
        scores.extend(batch_scores)

    return labels, scores

df['sentiment_label'], df['sentiment_score'] = apply_finbert_to_batch(df, 'headline_summary', batch_size=20)

In [6]:
import random

test_num = random.randint(1, 6000)
print(df.iloc[test_num].headline)
print(df.iloc[test_num].sentiment_label)
print(df.iloc[test_num].sentiment_score)

df

A Look Into NextEra Energy Partners Debt
Neutral
0.99933123588562


,headline,summary,created_at,ticker,headline_summary,tokens,filtered_tokens,lemmatized,impact_score,sentiment_label,sentiment_score
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again,2024-01-26,IBM,How We Missed A Winning Trade On IBM--And How ...,"['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...","['How', 'We', 'Missed', 'A', 'Winning', 'Trade...",0.0,Negative,0.967072
1,Investigating IBM's Standing In IT Services In...,NaN,2024-01-26,IBM,Investigating IBM's Standing In IT Services In...,"['Investigating', 'IBM', ""'s"", 'Standing', 'In...","['Investigating', 'IBM', ""'s"", 'Standing', 'In...","['Investigating', 'IBM', ""'s"", 'Standing', 'In...",0.0,Neutral,0.998521
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26,IBM,90's Tech Titans Are Outshining 'Magnificent S...,"['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...","['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...","['90', ""'s"", 'Tech', 'Titans', 'Are', 'Outshin...",1.2,Positive,1.000000
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25,IBM,Why Humana Shares Are Trading Lower By Over 11...,"['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...","['Why', 'Humana', 'Shares', 'Are', 'Trading', ...",1.9,Negative,0.999982
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...","['S', '&', 'P', '500', 'Eyes', 'Sixth', 'Green...",2.0,Positive,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
6798,"Earnings Scheduled For July 30, 2020",Companies Reporting Before The Bell • Masterca...,2020-07-30,TOT,"Earnings Scheduled For July 30, 2020 Companies...","['Earnings', 'Scheduled', 'For', 'July', '30',...","['Earnings', 'Scheduled', 'For', 'July', '30',...","['Earnings', 'Scheduled', 'For', 'July', '30',...",1.8,Neutral,0.999995
6799,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"Gainers • Laredo Petroleum, Inc. (NYSE:LPI) st...",2020-06-02,TOT,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...","['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...","['13', 'Energy', 'Stocks', 'Moving', 'In', 'Tu...",0.0,Positive,0.999999
6800,"Earnings Scheduled For May 5, 2020",Companies Reporting Before The Bell L3Harris T...,2020-05-05,TOT,"Earnings Scheduled For May 5, 2020 Companies R...","['Earnings', 'Scheduled', 'For', 'May', '5', '...","['Earnings', 'Scheduled', 'For', 'May', '5', '...","['Earnings', 'Scheduled', 'For', 'May', '5', '...",1.8,Neutral,0.999999
6801,"Earnings Scheduled For April 30, 2020",Companies Reporting Before The Bell McDonald&#...,2020-04-30,TOT,"Earnings Scheduled For April 30, 2020 Companie...","['Earnings', 'Scheduled', 'For', 'April', '30'...","['Earnings', 'Scheduled', 'For', 'April', '30'...","['Earnings', 'Scheduled', 'For', 'April', '30'...",1.8,Neutral,0.999999


In [5]:
# Save dataset after feature-engineered
df.to_csv('financial_news_wfeatures.csv', index=False)